In [1]:
%matplotlib inline


Gaussian HMM of accel_walk
--------------------------


In [1]:
from __future__ import print_function
import datetime
import json
from pprint import pprint
import numpy as np
from matplotlib import cm, pyplot as plt
from matplotlib.dates import YearLocator, MonthLocator
from hmmlearn.hmm import GaussianHMM


with open('/Users/DoerLBH/Dropbox/git/DREAM_PDBiomarker/test_walk_outbound.json') as data_file:    
    data = json.load(data_file)

timestamp = np.array(data[t]['timestamp'] for t in list(range(1, len(data))))
x = np.array(data[t]['x'] for t in list(range(1, len(data))))
y = np.array(data[t]['y'] for t in list(range(1, len(data))))
z = np.array(data[t]['z'] for t in list(range(1, len(data))))


In [ ]:

# Pack x,y,z for training.
X = np.column_stack([x, y, z])

Run Gaussian HMM



In [ ]:
print("fitting to HMM and decoding ...", end="")

startprob = np.array([0.6, 0.3, 0.1, 0.0])
# The transition matrix, note that there are no transitions possible
# between component 1 and 3
transmat = np.array([[0.7, 0.2, 0.0, 0.1],
                     [0.3, 0.5, 0.2, 0.0],
                     [0.0, 0.3, 0.5, 0.2],
                     [0.2, 0.0, 0.2, 0.6]])

# The means of each component
# means = np.array([[0.0,  0.0],
#                   [0.0, 11.0],
#                   [9.0, 10.0],
#                   [11.0, -1.0]])

# Build an HMM instance and set parameters
model = GaussianHMM(n_components=4, covariance_type="diag", n_iter=1000)
# Instead of fitting it from the data, we directly set the estimated
# parameters, the means and covariance of the components
model.startprob_ = startprob
model.transmat_ = transmat
model.means_ = means

# Make an HMM instance and execute fit
model.fit(X)

# Predict the optimal sequence of internal hidden state
hidden_states = model.predict(X)

print("done")

Print trained parameters and plot



In [ ]:
print("Transition matrix")
print(model.transmat_)
print()

print("Means and vars of each hidden state")
for i in range(model.n_components):
    print("{0}th hidden state".format(i))
    print("mean = ", model.means_[i])
    print("var = ", np.diag(model.covars_[i]))
    print()

fig, axs = plt.subplots(model.n_components, sharex=True, sharey=True)
colours = cm.rainbow(np.linspace(0, 1, model.n_components))
for i, (ax, colour) in enumerate(zip(axs, colours)):
    # Use fancy indexing to plot data in each state.
    mask = hidden_states == i
    ax.plot_date(dates[mask], close_v[mask], ".-", c=colour)
    ax.set_title("{0}th hidden state".format(i))

    # Format the ticks.
    ax.xaxis.set_major_locator(YearLocator())
    ax.xaxis.set_minor_locator(MonthLocator())

    ax.grid(True)

plt.show()